In [3]:
import pandas as pd
import numpy as np
import os
import sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from typing import Tuple, List


# Add project root to sys.path
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.db.base_utils import connect_db, disconnect_db

In [4]:
def prepare_goalie_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Prepare goalie data for modeling by calculating rolling averages and key metrics
    
    Args:
        df: DataFrame with goalie game-by-game stats
    Returns:
        Processed DataFrame with features for modeling
    """
    # Sort by goalie and date
    df = df.sort_values(['goalie_name', 'game_date'])
    
    # Calculate rolling averages (last 5 games) for key metrics
    rolling_stats = [
        'hd_save_pct', 'md_save_pct', 'ld_save_pct',
        'hd_shots_against', 'md_shots_against', 'ld_shots_against',
        'hd_goals_against', 'md_goals_against', 'ld_goals_against'
    ]
    
    for stat in rolling_stats:
        df[f'{stat}_rolling_5'] = df.groupby('goalie_name')[stat].rolling(5).mean().reset_index(0, drop=True)
    
    # Calculate danger-weighted save percentage
    df['danger_weighted_sv_pct'] = (
        (df['hd_saves'] * 3 + df['md_saves'] * 2 + df['ld_saves']) / 
        ((df['hd_shots'] * 3 + df['md_shots'] * 2 + df['ld_shots']))
    )
    
    return df

def create_goalie_features(df: pd.DataFrame) -> Tuple[pd.DataFrame, List[str]]:
    """
    Create feature set for goalie prediction model
    
    Args:
        df: Processed DataFrame with rolling stats
    Returns:
        Feature DataFrame and list of feature names
    """
    features = [
        'hd_save_pct_rolling_5',
        'md_save_pct_rolling_5', 
        'ld_save_pct_rolling_5',
        'hd_shots_against_rolling_5',
        'md_shots_against_rolling_5',
        'ld_shots_against_rolling_5',
        'danger_weighted_sv_pct'
    ]
    
    X = df[features].copy()
    
    # Handle missing values
    X = X.fillna(X.mean())
    
    return X, features

def train_goalie_model(X: pd.DataFrame, y: pd.Series) -> RandomForestRegressor:
    """
    Train Random Forest model for goalie performance prediction
    
    Args:
        X: Feature DataFrame
        y: Target variable (e.g. save percentage)
    Returns:
        Trained model
    """
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    model = RandomForestRegressor(
        n_estimators=100,
        max_depth=5,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    
    print(f"Train R2 Score: {train_score:.3f}")
    print(f"Test R2 Score: {test_score:.3f}")
    
    return model

def predict_goalie_performance(
    model: RandomForestRegressor,
    goalie_stats: pd.DataFrame,
    features: List[str]
) -> float:
    """
    Predict goalie performance for upcoming game
    
    Args:
        model: Trained model
        goalie_stats: DataFrame with goalie's recent stats
        features: List of feature names used in model
    Returns:
        Predicted save percentage
    """
    X_pred = goalie_stats[features].iloc[-1:]
    prediction = model.predict(X_pred)[0]
    return prediction

# Example usage:
# processed_data = prepare_goalie_data(goalie_df)
# X, features = create_goalie_features(processed_data)
# y = processed_data['save_pct']
# model = train_goalie_model(X, y)

In [5]:
def get_goalie_stats(
    goalie_name: str = None,
    team: str = None,
    start_date: str = None,
    end_date: str = None,
    db_prefix: str = "NST_DB_"
) -> pd.DataFrame:
    """
    Retrieve goalie stats from the database with optional filters.
    
    Args:
        goalie_name (str, optional): Name of specific goalie to retrieve
        team (str, optional): Team abbreviation to filter by
        start_date (str, optional): Start date in YYYY-MM-DD format
        end_date (str, optional): End date in YYYY-MM-DD format
        db_prefix (str): Database connection prefix
        
    Returns:
        pd.DataFrame: DataFrame containing goalie statistics
    """
    # Build the base query
    query = """
    SELECT *
    FROM goalie_stats
    WHERE 1=1
    """
    params = []
    
    # Add filters if provided
    if goalie_name:
        query += " AND player = %s"
        params.append(goalie_name)
    
    if team:
        query += " AND team = %s"
        params.append(team)
    
    if start_date:
        query += " AND date >= %s"
        params.append(start_date)
    
    if end_date:
        query += " AND date <= %s"
        params.append(end_date)
    
    # Add ordering
    query += " ORDER BY date DESC"
    
    try:
        # Connect to database
        conn = connect_db(db_prefix)
        
        # Execute query and load results into DataFrame
        df = pd.read_sql_query(query, conn, params=params)
        
        return df
        
    except Exception as e:
        logger.error(f"Error retrieving goalie stats: {e}")
        raise
    finally:
        if conn:
            disconnect_db(conn)

def get_goalie_rolling_stats(
    goalie_name: str,
    date: str,
    n_games: int = 5,
    db_prefix: str = "NST_DB_"
) -> pd.DataFrame:
    """
    Get rolling average stats for a specific goalie up to a given date.
    
    Args:
        goalie_name (str): Name of the goalie
        date (str): Date to get stats up to (YYYY-MM-DD)
        n_games (int): Number of games to include in rolling average
        db_prefix (str): Database connection prefix
        
    Returns:
        pd.DataFrame: DataFrame with rolling average stats
    """
    query = """
    WITH recent_games AS (
        SELECT *
        FROM goalie_stats
        WHERE player = %s
        AND date <= %s
        ORDER BY date DESC
        LIMIT %s
    )
    SELECT 
        player,
        team,
        AVG(sv_pct) as avg_sv_pct,
        AVG(gaa) as avg_gaa,
        AVG(gsaa) as avg_gsaa,
        AVG(hdsv_pct) as avg_hd_sv_pct,
        AVG(mdsv_pct) as avg_md_sv_pct,
        AVG(ldsv_pct) as avg_ld_sv_pct,
        AVG(hdgsaa) as avg_hd_gsaa,
        AVG(mdgsaa) as avg_md_gsaa,
        AVG(ldgsaa) as avg_ld_gsaa,
        COUNT(*) as games_played
    FROM recent_games
    GROUP BY player, team
    """
    
    try:
        conn = connect_db(db_prefix)
        df = pd.read_sql_query(query, conn, params=[goalie_name, date, n_games])
        return df
        
    except Exception as e:
        logger.error(f"Error retrieving rolling stats: {e}")
        raise
    finally:
        if conn:
            disconnect_db(conn)

def get_goalie_comparison(
    date: str,
    n_games: int = 5,
    min_games: int = 3,
    db_prefix: str = "NST_DB_"
) -> pd.DataFrame:
    """
    Get comparison stats for all goalies with recent activity.
    
    Args:
        date (str): Date to get stats up to (YYYY-MM-DD)
        n_games (int): Number of games to include in rolling averages
        min_games (int): Minimum games played to be included
        db_prefix (str): Database connection prefix
        
    Returns:
        pd.DataFrame: DataFrame with goalie comparison stats
    """
    query = """
    WITH recent_stats AS (
        SELECT 
            player,
            team,
            date,
            sv_pct,
            gaa,
            gsaa,
            hdsv_pct,
            mdsv_pct,
            ldsv_pct,
            hdgsaa,
            mdgsaa,
            ldgsaa,
            ROW_NUMBER() OVER (PARTITION BY player ORDER BY date DESC) as game_number
        FROM goalie_stats
        WHERE date <= %s
    )
    SELECT 
        player,
        team,
        AVG(sv_pct) as avg_sv_pct,
        AVG(gaa) as avg_gaa,
        AVG(gsaa) as avg_gsaa,
        AVG(hdsv_pct) as avg_hd_sv_pct,
        AVG(mdsv_pct) as avg_md_sv_pct,
        AVG(ldsv_pct) as avg_ld_sv_pct,
        AVG(hdgsaa) as avg_hd_gsaa,
        AVG(mdgsaa) as avg_md_gsaa,
        AVG(ldgsaa) as avg_ld_gsaa,
        COUNT(*) as games_played
    FROM recent_stats
    WHERE game_number <= %s
    GROUP BY player, team
    HAVING COUNT(*) >= %s
    ORDER BY avg_gsaa DESC
    """
    
    try:
        conn = connect_db(db_prefix)
        df = pd.read_sql_query(query, conn, params=[date, n_games, min_games])
        return df
        
    except Exception as e:
        logger.error(f"Error retrieving goalie comparison: {e}")
        raise
    finally:
        if conn:
            disconnect_db(conn)

# Example usage:
# Get recent stats for a specific goalie
"""
goalie_stats = get_goalie_stats(
    goalie_name="Adin Hill",
    start_date="2024-10-01",
    end_date="2024-10-15"
)

# Get rolling averages for a goalie
rolling_stats = get_goalie_rolling_stats(
    goalie_name="Adin Hill",
    date="2024-10-15",
    n_games=5
)

# Get comparison stats for all active goalies
comparison_stats = get_goalie_comparison(
    date="2024-10-15",
    n_games=5,
    min_games=3
)
"""

'\ngoalie_stats = get_goalie_stats(\n    goalie_name="Adin Hill",\n    start_date="2024-10-01",\n    end_date="2024-10-15"\n)\n\n# Get rolling averages for a goalie\nrolling_stats = get_goalie_rolling_stats(\n    goalie_name="Adin Hill",\n    date="2024-10-15",\n    n_games=5\n)\n\n# Get comparison stats for all active goalies\ncomparison_stats = get_goalie_comparison(\n    date="2024-10-15",\n    n_games=5,\n    min_games=3\n)\n'